<a href="https://colab.research.google.com/github/akshat-suwalka/Machine-Learning-Algorithms-from-Scratch/blob/master/K_means_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Library

In [1]:
import numpy as np
from numpy.linalg import norm

##Dataset

##K-means Algorithm from scratch

###Random Initializtion of Centroids

In [2]:
def intialize_centroids(n_clusters, X):
  random_indx = np.random.permutation(X.shape[0])
  random_indx = random_indx[:n_clusters]
  centroids = X[random_indx]
  return centroids

In [3]:
#Example
toy_data = np.random.rand(10,5)
toy_data

array([[0.57032615, 0.7945255 , 0.2017099 , 0.83177053, 0.25256721],
       [0.12231346, 0.46112294, 0.27788717, 0.15282793, 0.45708607],
       [0.45611462, 0.83621665, 0.88023941, 0.33381772, 0.29733438],
       [0.36454285, 0.5088095 , 0.22284329, 0.72461627, 0.34542298],
       [0.91274663, 0.8757186 , 0.12088078, 0.16794523, 0.8664994 ],
       [0.83807831, 0.85307847, 0.71767904, 0.01280469, 0.06715342],
       [0.10500549, 0.49351006, 0.74689272, 0.50354827, 0.91637508],
       [0.63764809, 0.48109429, 0.34342931, 0.70748955, 0.12895042],
       [0.85314474, 0.89156759, 0.36701657, 0.28074647, 0.29845309],
       [0.9715712 , 0.23502184, 0.83462122, 0.64097034, 0.53388453]])

In [4]:
n_clusters = int(3)
centroids = intialize_centroids(n_clusters,toy_data)
centroids

array([[0.85314474, 0.89156759, 0.36701657, 0.28074647, 0.29845309],
       [0.63764809, 0.48109429, 0.34342931, 0.70748955, 0.12895042],
       [0.10500549, 0.49351006, 0.74689272, 0.50354827, 0.91637508]])

### Compute distance between datapoints and centroids

In [5]:
def compute_distance(n_clusters, X, centroids):
  distance = np.zeros((X.shape[0], n_clusters))
  for i in range(n_clusters):
    each_cluster = np.square(norm(X - centroids[i], axis = 1))
    distance[:,i] = each_cluster

  return distance


In [6]:
#Example
distance = compute_distance(n_clusters, toy_data, centroids)
distance

array([[0.42246285, 0.15358263, 1.15272865],
       [0.76886858, 0.68558693, 0.55526584],
       [0.42691213, 0.61521517, 0.67052661],
       [0.60524807, 0.13704928, 0.71707889],
       [0.39978714, 1.11602198, 1.30553705],
       [0.24996491, 0.80501329, 1.62954531],
       [1.29393625, 1.10827466, 0.        ],
       [0.42632431, 0.        , 1.10827466],
       [0.        , 0.42632431, 1.29393625],
       [0.84892046, 0.58172223, 0.99063241]])

###Find nearest cluster of each data point

In [7]:
def find_nearest_cluster(distance):
  return np.argmin(distance, axis = 1)

In [8]:
#Example
labels = find_nearest_cluster(distance)
labels

array([1, 2, 0, 1, 0, 0, 2, 1, 0, 1])

###Compute new/fresh centroids 

In [9]:
def compute_centroids(n_clusters, labels, X):
  fresh_centroids = np.zeros((n_clusters, X.shape[1]))
  for i in range(n_clusters):
    fresh_centroids[i] = np.mean(X[labels == i], axis = 0)
  
  return fresh_centroids

In [10]:
#Example
fresh_centroids = compute_centroids(n_clusters, labels, toy_data)
fresh_centroids

array([[0.76502108, 0.86414533, 0.52145395, 0.19882853, 0.38236007],
       [0.63602207, 0.50486278, 0.40065093, 0.72621168, 0.31520628],
       [0.11365948, 0.4773165 , 0.51238995, 0.3281881 , 0.68673057]])

### Calculate error through sum of squares method

In [11]:
def compute_sse(n_clusters, X, centroids, labels):
  each_cluster_sse = np.zeros(n_clusters)
  for i in range(n_clusters):
    each_cluster_sse = np.square(norm(X[labels == i] - centroids[i], axis = 1))
  ans = np.sum(each_cluster_sse, axis = 0)
  return ans

In [12]:
error = compute_sse(n_clusters, toy_data, fresh_centroids, labels)
error

0.277632920409339

### Complete algorithm at once

In [13]:
class Kmeans:
    '''Implementing Kmeans algorithm.'''

    def __init__(self, n_clusters, max_iter=100, random_state=123):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state

    def initializ_centroids(self, X):
        np.random.RandomState(self.random_state)
        random_idx = np.random.permutation(X.shape[0])
        centroids = X[random_idx[:self.n_clusters]]
        return centroids

    def compute_centroids(self, X, labels):
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            centroids[k, :] = np.mean(X[labels == k, :], axis=0)
        return centroids

    def compute_distance(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            row_norm = norm(X - centroids[k, :], axis=1)
            distance[:, k] = np.square(row_norm)
        return distance

    def find_closest_cluster(self, distance):
        return np.argmin(distance, axis=1)

    def compute_sse(self, X, labels, centroids):
        distance = np.zeros(X.shape[0])
        for k in range(self.n_clusters):
            distance[labels == k] = norm(X[labels == k] - centroids[k], axis=1)
        return np.sum(np.square(distance))
    
    def fit(self, X):
        self.centroids = self.initializ_centroids(X)
        for i in range(self.max_iter):
            old_centroids = self.centroids
            distance = self.compute_distance(X, old_centroids)
            self.labels = self.find_closest_cluster(distance)
            self.centroids = self.compute_centroids(X, self.labels)
            if np.all(old_centroids == self.centroids):
                break
        self.error = self.compute_sse(X, self.labels, self.centroids)
    
    def predict(self, X):
        distance = self.compute_distance(X, old_centroids)
        return self.find_closest_cluster(distance)

##Testing through scratch algorithm

In [14]:
toy_data = np.random.rand(10,5)

In [15]:
print("Data : \n{}".format(toy_data))
km = Kmeans(n_clusters=2, max_iter=100)
km.fit(toy_data)
centroids = km.centroids
print("\nCentroids : \n{}".format(centroids))
error = km.error
print("\nSSE_Error : \n{}".format(error))

Data : 
[[0.90176752 0.89509561 0.3089601  0.11503485 0.46502798]
 [0.89081707 0.26219081 0.85786181 0.79477776 0.65926119]
 [0.64274047 0.13623718 0.52647662 0.51682436 0.77440541]
 [0.2965551  0.77506156 0.3160154  0.509545   0.80053618]
 [0.07370216 0.64961757 0.74745577 0.89896963 0.98584538]
 [0.16333892 0.77482354 0.64230954 0.79585249 0.55805916]
 [0.42482017 0.18631904 0.32540633 0.43459994 0.58921947]
 [0.51885525 0.00244423 0.93968292 0.54121889 0.88636333]
 [0.51332459 0.56394495 0.37225844 0.04803221 0.84409154]
 [0.91769163 0.75455459 0.34014141 0.50500245 0.7120671 ]]

Centroids : 
[[0.6108318  0.63499515 0.33255634 0.32244289 0.68218845]
 [0.45789078 0.36506267 0.74275733 0.70952863 0.77278689]]

SSE_Error : 
2.1735351324378134


##Testing thorugh Sklearn

In [16]:
from sklearn.cluster import KMeans

In [17]:
print("Data : \n{}".format(toy_data))
km = Kmeans(n_clusters=2, max_iter=100)
km.fit(toy_data)
centroids = km.centroids
print("\nCentroids : \n{}".format(centroids))
error = km.error
print("\nSSE_Error : \n{}".format(error))

Data : 
[[0.90176752 0.89509561 0.3089601  0.11503485 0.46502798]
 [0.89081707 0.26219081 0.85786181 0.79477776 0.65926119]
 [0.64274047 0.13623718 0.52647662 0.51682436 0.77440541]
 [0.2965551  0.77506156 0.3160154  0.509545   0.80053618]
 [0.07370216 0.64961757 0.74745577 0.89896963 0.98584538]
 [0.16333892 0.77482354 0.64230954 0.79585249 0.55805916]
 [0.42482017 0.18631904 0.32540633 0.43459994 0.58921947]
 [0.51885525 0.00244423 0.93968292 0.54121889 0.88636333]
 [0.51332459 0.56394495 0.37225844 0.04803221 0.84409154]
 [0.91769163 0.75455459 0.34014141 0.50500245 0.7120671 ]]

Centroids : 
[[0.68714524 0.40011234 0.52439823 0.42221292 0.704348  ]
 [0.17786539 0.73316756 0.56859357 0.73478904 0.78148024]]

SSE_Error : 
2.2305559694885955
